In [3]:
import warnings
warnings.filterwarnings(action='ignore')
import pymongo
import requests

In [4]:
# mongoDB에 저장된 시도 목록을 얻어온다.
mongoConnection = pymongo.MongoClient('mongodb://localhost:27017/')
starbucks = mongoConnection['starbucks'] 
sido = starbucks['sido']
sidoList = sido.find({}, {'_id': 0})

시도 이름을 입력받아 시도 이름에 해당하는 시도 코드를 sido 컬렉션에서 얻어온 후 크롤링하기

In [5]:
for s in sidoList:
    print(s['sido_nm'], end=' ')

서울 경기 광주 대구 대전 부산 울산 인천 강원 경남 경북 전남 전북 충남 충북 제주 세종 서울 서울 서울 서울 서울 서울 서울 서울 서울 서울 서울 서울 서울 서울 서울 서울 서울 서울 서울 서울 서울 서울 서울 서울 서울 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 광주 광주 광주 광주 광주 대구 대구 대구 대구 대구 대구 대구 대구 대전 대전 대전 대전 대전 부산 부산 부산 부산 부산 부산 부산 부산 부산 부산 부산 부산 부산 부산 부산 부산 울산 울산 울산 울산 울산 인천 인천 인천 인천 인천 인천 인천 인천 인천 인천 강원 강원 강원 강원 강원 강원 강원 강원 강원 강원 강원 강원 강원 강원 강원 강원 강원 강원 경남 경남 경남 경남 경남 경남 경남 경남 경남 경남 경남 경남 경남 경남 경남 경남 경남 경남 경남 경남 경북 경북 경북 경북 경북 경북 경북 경북 경북 경북 경북 경북 경북 경북 경북 경북 경북 경북 경북 경북 경북 경북 경북 경북 전남 전남 전남 전남 전남 전남 전남 전남 전남 전남 전남 전남 전남 전남 전남 전남 전남 전남 전남 전남 전남 전남 전북 전북 전북 전북 전북 전북 전북 전북 전북 전북 전북 전북 전북 전북 전북 충남 충남 충남 충남 충남 충남 충남 충남 충남 충남 충남 충남 충남 충남 충남 충남 충남 충북 충북 충북 충북 충북 충북 충북 충북 충북 충북 충북 충북 충북 충북 제주 제주 세종 서울 서울 서울 서울 서울 서울 서울 서울 서울 서울 서울 서울 서울 서울 서울 서울 서울 서울 서울 서울 서울 서울 서울 서울 서울 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 경기 광

In [6]:
sidoName = input('구군코드를 검색할 시도 이름을 입력하세요(ex. 서울): ')
# sido 컬렉션에서 입력받은 시도 이름에 해당하는 document 1건을 얻어온다.
sido_cd = sido.find({'sido_nm': sidoName}, {'_id': 0})
print(sido_cd.next())

구군코드를 검색할 시도 이름을 입력하세요(ex. 서울): 서울
{'sido_cd': '01', 'sido_nm': '서울'}


시도 이름에 따른 전체 구군을 컬렉션에 document로 저장하기

In [7]:
gugun = starbucks['gugun']
gugun.delete_many({}) # gugun 컬렉션에 저장된 모든 데이터를 제거한다.
# starbucks 데이터베이스의 sido 컬렉션에 저장된 도큐먼트 개수만큼 반복하며 모든 구군을 gugun 컬렉션에 저장한다.
gugunList = []
sidoList = sido.find({}, {'_id': 0})
for s in sidoList:
    request = requests.post('https://www.starbucks.co.kr/store/getGugunList.do', data={
        'sido_cd': s['sido_cd']
    }).json()
    for r in request['list']:
        # 세종 처리
        if r['gugun_nm'] is None:
            r['gugun_nm'] = ''
        d = {'sido_cd': s['sido_cd'].strip(), 'sido_nm': s['sido_nm'].strip(), 'gugun_cd': r['gugun_cd'].strip(), 'gugun_nm': r['gugun_nm'].strip()}
        gugunList.append(d)

gugunAll = starbucks['gugunAll'] # gugunAll 컬렉션을 만든다.
gugunAll.delete_many({}) # gugunAll 컬렉션에 저장된 모든 데이터를 제거한다.
for g in gugunList:
    gugunAll.insert_one(g)

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, '현재 연결은 원격 호스트에 의해 강제로 끊겼습니다', None, 10054, None))